In [356]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pymol import cmd
import sys, db_utils, re, os, ast, requests
sys.path.append('/Users/chopyanlee/Coding/Python/DMI/iupred2a/')
import iupred2a_lib
connect = db_utils.get_connection()

In [357]:
# Load in the previously obtained full-length sequences of the proteins carrying domains forming a DDI in the
# manual curation set as a DataFrame
df = pd.read_csv('/Users/chopyanlee/Coding/Python/DDI/DDI_type_manual_curation/new_protein_sequences_for_IUPred_for_John.csv',
                index_col=0)
df.reset_index(drop=True,inplace=True)
df

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,...,Motif_binding_domain,DMI_overlap_checked,DMI_overlap_identified,DDI_approved,Curator,Comments,non-approved category,Unnamed: 19,Uniprot_seq1,Uniprot_seq2
0,PF00004_PF10584,6EPF,I,PF00004,AAA,222,355,C,PF10584,Proteasome_A_N,...,NaN,NaN,NaN,0.0,"JG, KL",1 out of 1 supporting CS showed an interchain ...,HMM (at least one Pfam has unfolded structure),linear stretch within domain,MGQSQSGGHGPGGGKKDDKDKKKKYEPPVPTRVGKKKKKTKGPDAA...,MSRRYDSRTTIFSPEGRLYQVEYAMEAIGHAGTCLGILANDGVLLA...
1,PF00009_PF01873,2D74,A,PF00009,GTP_EFTU,9,201,B,PF01873,eIF-5_eIF-2B,...,NaN,NaN,NaN,1.0,JG,11 out of 11 supporting CS showed interchain i...,NaN,NaN,MGEKRKTRQAEVNIGMVGHVDHGKTTLTKALTGVWTDTHSEELRRG...,MEIDYYDYEKLLEKAYQELPENVKHHKSRFEVPGALVTIEGNKTII...
2,PF00010_PF02344,6G6L,B,PF00010,HLH,214,254,C,PF02344,Myc-LZ,...,NaN,NaN,NaN,0.0,"JG, KL",2 out of 6 supporting structures in 3did showe...,crystal contact,NaN,MSDNDDIEVESDEEQPRFQSAADKRAHHNALERKRRDHIKDSFHSL...,MDFFRVVENQQPPATMPLNVSFTNRNYDLDYDSVQPYFYCDEEENF...
3,PF00019_PF00041,4UI2,B,PF00019,TGF_beta,294,395,A,PF00041,fn3,...,NaN,NaN,NaN,0.0,JG,Only 1 supporting CS was present for the DDI t...,crystal contact,study describes crystal contact interface,MVAGTRCLLALLLPQVLLGGAAGLVPELGRRKFAAASSGRPSSQPS...,MAAERGARRLLSTPSFWLYCLLLLGRRAPGAAAARSGSAPQSPGAS...
4,PF00023_PF07686,4NIK,A,PF00023,Ank,105,137,B,PF07686,V-set,...,PF00023,1.0,0.0,0.0,JL,Only one supporting structure was present in 3...,synthetic construct,NaN,MEGCVSNLMVCNLAYSGKLEELKESILADKSLATRTDQDSRTALHW...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PF16004_PF00400,6ID0,C,PF16004,EFTUD2,56,110,T,PF00400,WD40,...,PF16004,1.0,0.0,0.0,"JG, KL",10 out of 10 supporting CS displayed interchai...,HMM (at least one Pfam has unfolded structure),linear stretch within the domain (whole domain...,MDTDLYDEFGNYIGPELDSDEDDDELGRETKDLDEMDDDDDDDDVG...,MVEEVQKHSVHTLVFRSLKRTHDMFVADNGKPVPLDEESHKRKMAI...
76,PF17292_PF08644,4KHB,H,PF17292,POB3_N,5,97,G,PF08644,SPT16,...,NaN,NaN,NaN,0.0,"JG, KL",3 out of 3 supporting CS displayed interchain ...,contact is minimal,NaN,MAAIESFDHIYLDLSKEPGKCRFAENGLGWKPVGGGETFTLDVSNI...,MADIKIDSKLFQERISHFISAWKADKRSGDALFGGVSSIVILMGKV...
77,PF17820_PF16523,3L4F,D,PF17820,PDZ_6,701,755,A,PF16523,betaPIX_CC,...,NaN,NaN,NaN,0.0,"JG, JL",Only one structure supporting the DDI type (Sc...,HMM (at least one Pfam has unfolded structure),motif is C-terminally included in the Pfam match,MTHSPATSEDEERHSASECPEGGSESDSSPDGPGRGPQGTRGRGSG...,MTDNANSQLVVRAKFNFQQTNEDELSFSKGDVIHVTRVEEGGWWEG...
78,PF17838_PF00071,3KZ1,B,PF17838,PH_16,950,1079,E,PF00071,Ras,...,NaN,NaN,NaN,1.0,JG,9 out of 9 supporting CS displayed interchain ...,NaN,NaN,MSVRLPQSIDRLSSLSSLGDSAPERKSPSHHRQPSDASETTGLVQR...,MAAIRKKLVIVGDGACGKTCLLIVFSKDQFPEVYVPTVFENYVADI...


In [358]:
# Download the PDB structures from the maunal curation set that will be needed later
structure_path = '/Users/chopyanlee/Coding/Python/DDI/curated_structures/'
for pdb_id in df['PDB_ID'].to_list():
    cmd.cd(structure_path)
    if not os.path.exists(f'{structure_path}{pdb_id}.cif'):
        cmd.fetch(pdb_id,type='cif')
    cmd.reinitialize()

In [359]:
# Perform the IUPred calculation using the crystallized sequence
df['chain1_cryst_domain_seq'] = np.nan
df['chain2_cryst_domain_seq'] = np.nan
df['chain1_cryst_avg_iface_iupred'] = np.nan
df['chain2_cryst_avg_iface_iupred'] = np.nan
df['chain1_cryst_frac_iface_above04'] = np.nan
df['chain2_cryst_frac_iface_above04'] = np.nan

for i, r in df.iterrows():
    print(i)
    cmd.cd(structure_path)
    pdb_id = r['PDB_ID']
    cmd.load(f'{structure_path}{pdb_id}.pdb')
    chain1 = r['Chain_ID1']
    start1 = r['DomainStart1']
    end1 = r['DomainEnd1']
    chain1_intf = r['Chain1Iface'].split(',')
    chain1_intf_resi = set([ele.split(':')[-1] for ele in r['Chain1Iface'].split(',')])
    chain1_domain_dict = {}
    chain2 = r['Chain_ID2']
    start2 = r['DomainStart2']
    end2 = r['DomainEnd2']
    chain2_intf = r['Chain2Iface'].split(',')
    chain2_intf_resi = set([ele.split(':')[-1] for ele in r['Chain2Iface'].split(',')])
    chain2_domain_dict = {}
    cmd.iterate(f'{pdb_id} and chain {chain1} and resi \\{start1}-\\{end1} and n. CA',
                'chain1_domain_dict[resi]=oneletter', space={'chain1_domain_dict':chain1_domain_dict})
    cmd.iterate(f'{pdb_id} and chain {chain2} and resi \\{start2}-\\{end2} and n. CA',
                'chain2_domain_dict[resi]=oneletter', space={'chain2_domain_dict':chain2_domain_dict})
    chain1_iupred_dict = {}
    chain2_iupred_dict = {}
    for domain_dict, iupred_dict in zip([chain1_domain_dict,chain2_domain_dict],[chain1_iupred_dict,chain2_iupred_dict]):
        domain_seq = ''.join(domain_dict.values())
        iupred = iupred2a_lib.iupred(domain_seq,mode='short')
        for resi, iupred in zip(domain_dict.keys(),iupred):
            iupred_dict.update({resi:iupred})
    df.loc[i,'chain1_cryst_domain_seq'] = ''.join(chain1_domain_dict.values())
    df.loc[i,'chain2_cryst_domain_seq'] = ''.join(chain2_domain_dict.values())
    df.loc[i,'chain1_cryst_avg_iface_iupred'] = np.mean([chain1_iupred_dict.get(resi) for resi in chain1_intf_resi])
    df.loc[i,'chain2_cryst_avg_iface_iupred'] = np.mean([chain2_iupred_dict.get(resi) for resi in chain2_intf_resi])
    df.loc[i,'chain1_cryst_frac_iface_above04'] = np.sum(np.array([chain1_iupred_dict.get(resi) for resi in chain1_intf_resi])>= 0.4) / len(chain1_intf_resi)
    df.loc[i,'chain2_cryst_frac_iface_above04'] = np.sum(np.array([chain2_iupred_dict.get(resi) for resi in chain2_intf_resi])>= 0.4) / len(chain2_intf_resi)
    cmd.reinitialize()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [360]:
df

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,...,non-approved category,Unnamed: 19,Uniprot_seq1,Uniprot_seq2,chain1_cryst_domain_seq,chain2_cryst_domain_seq,chain1_cryst_avg_iface_iupred,chain2_cryst_avg_iface_iupred,chain1_cryst_frac_iface_above04,chain2_cryst_frac_iface_above04
0,PF00004_PF10584,6EPF,I,PF00004,AAA,222,355,C,PF10584,Proteasome_A_N,...,HMM (at least one Pfam has unfolded structure),linear stretch within domain,MGQSQSGGHGPGGGKKDDKDKKKKYEPPVPTRVGKKKKKTKGPDAA...,MSRRYDSRTTIFSPEGRLYQVEYAMEAIGHAGTCLGILANDGVLLA...,VILYGPPGTGKTLLAKAVANQTSATFLRVVGSELIQKYLGDGPKLV...,YDSRTTIFSPEGRLYQVEYAMEA,0.284420,0.343450,0.400000,0.166667
1,PF00009_PF01873,2D74,A,PF00009,GTP_EFTU,9,201,B,PF01873,eIF-5_eIF-2B,...,NaN,NaN,MGEKRKTRQAEVNIGMVGHVDHGKTTLTKALTGVWTDTHSEELRRG...,MEIDYYDYEKLLEKAYQELPENVKHHKSRFEVPGALVTIEGNKTII...,QAEVNIGMVGHVDHGKTTLTKALTGVWTDTLRRGITIKIGFADAEI...,PENVKHHKSRFEVPGALVTIEGNKTIIENFKDIADALNRDPQHLLK...,0.101112,0.152533,0.000000,0.133333
2,PF00010_PF02344,6G6L,B,PF00010,HLH,214,254,C,PF02344,Myc-LZ,...,crystal contact,NaN,MSDNDDIEVESDEEQPRFQSAADKRAHHNALERKRRDHIKDSFHSL...,MDFFRVVENQQPPATMPLNVSFTNRNYDLDYDSVQPYFYCDEEENF...,RRDHIKDSFHSLRDSVPSLQGEKASRAQILDKATEYIQYMR,AEEQKLISEEDLLRKRREQLKHKLEQLRNS,0.480833,0.490229,1.000000,0.571429
3,PF00019_PF00041,4UI2,B,PF00019,TGF_beta,294,395,A,PF00041,fn3,...,crystal contact,study describes crystal contact interface,MVAGTRCLLALLLPQVLLGGAAGLVPELGRRKFAAASSGRPSSQPS...,MAAERGARRLLSTPSFWLYCLLLLGRRAPGAAAARSGSAPQSPGAS...,SSCKRHPLYVDFSDVGWNDWIVAPPGYHAFYCHGECPFPLADHLNS...,MPPVGVQASILSHDTIRITWADNSLPKHQKITDSRYYTVRWKTNIP...,0.046050,0.271160,0.000000,0.100000
4,PF00023_PF07686,4NIK,A,PF00023,Ank,105,137,B,PF07686,V-set,...,synthetic construct,NaN,MEGCVSNLMVCNLAYSGKLEELKESILADKSLATRTDQDSRTALHW...,NaN,NGCTPLHYAASKNRHEIAVMLLEGGANPDAKDH,ESGGSLVKPGGSLRLSCAASGFTFSNYSMNWVRQAPGKGLEWISSI...,0.410000,0.057314,0.666667,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PF16004_PF00400,6ID0,C,PF16004,EFTUD2,56,110,T,PF00400,WD40,...,HMM (at least one Pfam has unfolded structure),linear stretch within the domain (whole domain...,MDTDLYDEFGNYIGPELDSDEDDDELGRETKDLDEMDDDDDDDDVG...,MVEEVQKHSVHTLVFRSLKRTHDMFVADNGKPVPLDEESHKRKMAI...,EVVLHEDKKYYPTAEEVYGPEVETIVQEEDTQPLTEPIIKPVKTKK...,WKLYRVISGHLGWVRCIAVEPGNQWFVTGSADRTIKIWD,0.498730,0.268783,0.900000,0.250000
76,PF17292_PF08644,4KHB,H,PF17292,POB3_N,5,97,G,PF08644,SPT16,...,contact is minimal,NaN,MAAIESFDHIYLDLSKEPGKCRFAENGLGWKPVGGGETFTLDVSNI...,MADIKIDSKLFQERISHFISAWKADKRSGDALFGGVSSIVILMGKV...,ESFDHIYLDLSKEPGKCRFAENGLGWKPVGGTFTLDVSNIGGAQWS...,IVIDQKNNTIVLPIMGRPVPFHINTIKNASKSDEGEWSFLRINFLS...,0.168683,0.181533,0.000000,0.000000
77,PF17820_PF16523,3L4F,D,PF17820,PDZ_6,701,755,A,PF16523,betaPIX_CC,...,HMM (at least one Pfam has unfolded structure),motif is C-terminally included in the Pfam match,MTHSPATSEDEERHSASECPEGGSESDSSPDGPGRGPQGTRGRGSG...,MTDNANSQLVVRAKFNFQQTNEDELSFSKGDVIHVTRVEEGGWWEG...,YLESVDEGGVAWRAGLRMGDFLIEVNGQNVVKVGHRQVVNMIRQGG...,MEKSLVDTVYALKDEVQELRQDNKKMKKSLEEEQRARKDLEKLVRK...,0.338850,0.880200,0.250000,1.000000
78,PF17838_PF00071,3KZ1,B,PF17838,PH_16,950,1079,E,PF00071,Ras,...,NaN,NaN,MSVRLPQSIDRLSSLSSLGDSAPERKSPSHHRQPSDASETTGLVQR...,MAAIRKKLVIVGDGACGKTCLLIVFSKDQFPEVYVPTVFENYVADI...,NPLAAEFKSLDLTTRKMIHEGPLTWRISKDKTLDLHVLLLEDLLVL...,KLVIVGDGACGKTCLLIVFSKDQFPEVYVPTVFENYVADIEVDGKQ...,0.042713,0.178792,0.000000,0.166667


In [367]:
# Perform the IUPred calculation using the full length uniprot sequence, if available
df['chain1_uniprot_avg_iface_iupred'] = np.nan
df['chain2_uniprot_avg_iface_iupred'] = np.nan
df['chain1_uniprot_frac_iface_above04'] = np.nan
df['chain2_uniprot_frac_iface_above04'] = np.nan
df['chain1_num_iface_res'] = np.nan
df['chain2_num_iface_res'] = np.nan
df['chain1_unmapped_iface_res'] = np.nan
df['chain2_unmapped_iface_res'] = np.nan
structure_path = '/Users/chopyanlee/Coding/Python/DDI/curated_structures/'

for i, r in df.iterrows():
    print(i)
    pdb_id = r['PDB_ID'].lower()
    uniprot_seq1 = r['Uniprot_seq1']
    uniprot_seq2 = r['Uniprot_seq2']
    chain_id1 = r['Chain_ID1']
    chain_id2 = r['Chain_ID2']
    chain1_intf = r['Chain1Iface'].split(',')
    chain1_intf_resi = list(set([ele.split(':')[-1] for ele in r['Chain1Iface'].split(',')]))
    df.loc[i,'chain1_num_iface_res'] = len(chain1_intf_resi)
    chain2_intf = r['Chain2Iface'].split(',')
    chain2_intf_resi = list(set([ele.split(':')[-1] for ele in r['Chain2Iface'].split(',')]))
    df.loc[i,'chain2_num_iface_res'] = len(chain2_intf_resi)
    
    unp_ind_dict = {chain_id1:[],chain_id2:[]}
    url = f"https://www.ebi.ac.uk/pdbe/api/mappings/uniprot/{pdb_id.lower()}"
    response = requests.get(url)
    data = response.json()
    for k, v in data.get(pdb_id.lower()).get('UniProt').items():
        mappings = v.get('mappings')
        for mapping in mappings:
            chain_id = mapping.get('chain_id')
            unp_start = mapping.get('unp_start')
            unp_end = mapping.get('unp_end')
            pdb_start = mapping.get('start').get('residue_number') # not the resi, literally the index of the seq
            pdb_end = mapping.get('end').get('residue_number') # not the resi, literally the index of the seq
            if chain_id in unp_ind_dict.keys():
                print(unp_ind_dict)
                print(chain_id,unp_start,unp_end, pdb_start, pdb_end)
                # some pdb sequences match to uniprot sequence in fragments
                unp_ind_dict[chain_id].append([unp_start,unp_end, pdb_start, pdb_end])
                
    cmd.cd(structure_path)
    cmd.load(f'{structure_path}{pdb_id.upper()}.cif') # I observed in some cases cif shows unsolved residues that
    # SIFTS map onto that pdb structures do not show
    # for it to work with cmd.iterate the unsolved residues have to be in the file otherwise mapping is wrong
    pdb_ind_dict = {chain_id1:[], chain_id2:[]}
    for chain_id in pdb_ind_dict.keys():
        cmd.iterate(f'{pdb_id.upper()} and chain {chain_id} and n. CA','pdb_ind_dict.get(chain).append(resi)',
                   space={'pdb_ind_dict':pdb_ind_dict})
    cmd.reinitialize()
    
    pdb_2_unp = {}
    for chain_id in pdb_ind_dict.keys():
        if any(unp_ind_dict.get(chain_id)):
            pdb_ind = pdb_ind_dict.get(chain_id)
            if chain_id not in pdb_2_unp:
                pdb_2_unp[chain_id] = {}
            for mapping in unp_ind_dict.get(chain_id):
                mapped_pdb_ind = pdb_ind[mapping[2]-1:mapping[3]]
                pdb_2_unp[chain_id].update(dict(zip(mapped_pdb_ind,range(mapping[0],mapping[1]+1))))
    
    unp_iupred = {chain_id1:'', chain_id2:''}
    for chain_id, seq in zip([chain_id1, chain_id2],[uniprot_seq1, uniprot_seq2]):
        if pd.isna(seq):
            continue
        iupred_dict = {}
        iupred = iupred2a_lib.iupred(seq,mode='short')
        for resi, iupred in zip(range(len(seq)),iupred):
            iupred_dict.update({resi+1:iupred})
        unp_iupred[chain_id] = iupred_dict
    
    unp_intf_iupred = {chain_id1:'',chain_id2:''}
    for chain_id, chain_intf_resi in zip([chain_id1, chain_id2],[chain1_intf_resi, chain2_intf_resi]):
        pdb_2_unp_dict = pdb_2_unp.get(chain_id)
        iupred_dict = unp_iupred.get(chain_id)
        if pdb_2_unp.get(chain_id) is not None:
            unp_intf_resi = [pdb_2_unp_dict.get(resi) for resi in chain_intf_resi]
            unp_intf_resi_iupred = [iupred_dict.get(resi) for resi in unp_intf_resi]
            if None in unp_intf_resi:
                print(chain_id, chain_intf_resi)
                print(pdb_2_unp_dict)
                print(unp_intf_resi)
                print(unp_intf_resi_iupred)
            unp_intf_iupred[chain_id] = unp_intf_resi_iupred
    
    for ind, chain_id in enumerate(unp_intf_iupred):
        unp_intf_resi_iupred = unp_intf_iupred.get(chain_id)
        if any(unp_intf_resi_iupred):
            df.loc[i,f'chain{ind+1}_unmapped_iface_res'] = int(unp_intf_resi_iupred.count(None))
            if None in unp_intf_resi_iupred:
                unp_intf_resi_iupred = [ele for ele in unp_intf_resi_iupred if ele is not None]
            df.loc[i,f'chain{ind+1}_uniprot_avg_iface_iupred'] = np.mean(unp_intf_resi_iupred)
            df.loc[i,f'chain{ind+1}_uniprot_frac_iface_above04'] = np.sum(np.array(unp_intf_resi_iupred) >= 0.4) / len(unp_intf_resi_iupred)

0
{'I': [], 'C': []}
C 1 261 1 261
{'I': [], 'C': [[1, 261, 1, 261]]}
I 1 440 1 440
1
{'A': [], 'B': []}
A 1 411 1 411
{'A': [[1, 411, 1, 411]], 'B': []}
B 1 140 1 140
2
{'B': [], 'C': []}
C 351 437 8 94
{'B': [], 'C': [[351, 437, 8, 94]]}
B 22 103 2 83
3
{'B': [], 'A': []}
A 852 1102 4 254
{'B': [], 'A': [[852, 1102, 4, 254]]}
B 283 396 1 114
4
{'A': [], 'B': []}
A 1 226 5 230
5
{'B': [], 'C': []}
C 32 249 6 223
6
{'A': [], 'B': []}
A 60 385 1 326
{'A': [[60, 385, 1, 326]], 'B': []}
B 21 169 1 149
7
{'B': [], 'A': []}
A 501 771 13 283
{'B': [], 'A': [[501, 771, 13, 283]]}
B 1909 2037 2 130
8
{'E': [], 'I': []}
E 27 266 1 240
{'E': [[27, 266, 1, 240]], 'I': []}
I 61 117 1 57
9
{'i': [], 'p': []}
i 1 155 1 155
{'i': [[1, 155, 1, 155]], 'p': []}
p 1 350 1 350
10
{'B': [], 'C': []}
B 7 330 6 329
{'B': [[7, 330, 6, 329]], 'C': []}
C 417 583 4 170
11
{'y': [], 'W': []}
W 1 181 1 181
{'y': [], 'W': [[1, 181, 1, 181]]}
y 1 346 1 346
12
{'J': [], 'H': []}
H 6 170 6 170
{'J': [], 'H': [[6, 170,

{'C': [], 'A': []}
A 146 473 6 333
{'C': [], 'A': [[146, 473, 6, 333]]}
C 342 488 4 150
41
{'P': [], 'M': []}
M 1 282 1 282
{'P': [], 'M': [[1, 282, 1, 282]]}
P 1 317 1 317
42
{'o': [], 'l': []}
l 1 186 1 186
{'o': [], 'l': [[1, 186, 1, 186]]}
o 1 137 1 137
43
{'z': [], 'D': []}
D 1 306 1 306
{'z': [], 'D': [[1, 306, 1, 306]]}
z 1 278 1 278
44
{'A': [], 'B': []}
A 51 131 1 81
{'A': [[51, 131, 1, 81]], 'B': []}
B 932 1013 1 82
45
{'C': [], 'A': []}
A 1 554 1 554
{'C': [], 'A': [[1, 554, 1, 554]]}
C 1 99 1 99
46
{'K': [], 'A': []}
A 1 2335 1 2335
{'K': [], 'A': [[1, 2335, 1, 2335]]}
K 1 439 1 439
47
{'J': [], 'F': []}
F 1 330 2 331
{'J': [], 'F': [[1, 330, 2, 331]]}
J 1 133 4 136
48
{'A': [], 'B': []}
A 35 294 2 261
{'A': [[35, 294, 2, 261]], 'B': []}
B 17 112 1 96
49
{'D': [], 'i': []}
D 1 227 1 227
{'D': [[1, 227, 1, 227]], 'i': []}
i 24 133 1 110
50
{'D': [], 'B': []}
D 240 447 1 208
51
{'E': [], 'A': []}
A 1 447 19 465
{'E': [], 'A': [[1, 447, 19, 465]]}
E 1 260 1 260
52
{'A': [], 'B

In [371]:
df.to_csv('/Users/chopyanlee/Coding/Python/DDI/DDI_type_manual_curation/new_protein_sequences_for_IUPred_for_John_for_Johanna.csv')